In [156]:
!pip install tfds-nightly
!pip install tensorflow-hub

In [37]:
import os
import numpy as np
import time
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load compressed models from tensorflow_hub
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is NOT AVAILABLE


In [63]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [66]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(2000)))
y=''
# print(train_data.)
for x in train_examples_batch:
  # y= x.numpy()
  # y=y+' '
  # print(x)
  y=y+x.numpy().decode("utf-8")
  y=y+'\n'
print('Length of text: {} characters'.format(len(y)))

Length of text: 2597387 characters


In [67]:
# The unique characters in the file
vocab = sorted(set(y))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [69]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in y])

In [70]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\x08':   0,
  '\n':   1,
  '\x10':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '$' :   7,
  '%' :   8,
  '&' :   9,
  "'" :  10,
  '(' :  11,
  ')' :  12,
  '*' :  13,
  '+' :  14,
  ',' :  15,
  '-' :  16,
  '.' :  17,
  '/' :  18,
  '0' :  19,
  ...
}


In [71]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(y[:13]), text_as_int[:13]))

'This was an a' ---- characters mapped to int ---- > [55 75 76 86  3 90 68 86  3 68 81  3 68]


In [72]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(y)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
h
i
s
 


In [73]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. B"
'oth are great actors, but this must simply be their worst role in history. Even their great acting co'
"uld not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece"
'. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutio'
'ns. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pa'


In [74]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [75]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. "
Target data: "his was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. B"


In [76]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 55 ('T')
  expected output: 75 ('h')
Step    1
  input: 75 ('h')
  expected output: 76 ('i')
Step    2
  input: 76 ('i')
  expected output: 86 ('s')
Step    3
  input: 86 ('s')
  expected output: 3 (' ')
Step    4
  input: 3 (' ')
  expected output: 90 ('w')


In [77]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [78]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [79]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [80]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [81]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [82]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           33536     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 131)           134275    
Total params: 4,106,115
Trainable params: 4,106,115
Non-trainable params: 0
_________________________________________________________________


In [83]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [84]:
sampled_indices

array([ 17, 107, 112,   1,   5,  72,  36,  94,  25,  57,  25,  23, 119,
        89,  87,  72,  87,  94,  86,  88, 107,  41,  12,  25,  50, 105,
        61,   8, 119, 123, 129, 111,  32, 107,  47,   9, 109,  42,  90,
        22,  24, 130,  43,   8,  27,  65, 121, 101, 102,  89, 109,  54,
       100, 117,  14,  83,   1, 102,  24,  44,   0,  41,  97,  25,  80,
        95, 108,  10,  53,  82,  56, 122, 108,  92,  97,  97,  14,  97,
        23, 123, 115,  65, 112,  52,  67,  64, 110,  70, 108,  97,  75,
        15,   6,  32,  11,  53,  97,  59,  51, 116])

In [85]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'his one, especially by Gene Kelly. Two sailors go on liberty to see if they can find love and romanc'

Next Char Predictions: 
 '.¦Ê\n"eA{6V64çvtet{su¦F)6O¡Z%çëü´=¦L&¨Gw35ýH%8^é\x96\x97v¨S\x91ä+p\n\x975I\x08F\x806m}§\'RoUê§y\x80\x80+\x804ëâ^ÊQ`]®c§\x80h,#=(R\x80XPã'


In [86]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8742967


In [87]:
model.compile(optimizer='adam', loss=loss)

In [88]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [89]:
EPOCHS = 5

In [90]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
401/401 [==============================] - 1989s 5s/step - loss: 2.3047
Epoch 2/5
401/401 [==============================] - 1953s 5s/step - loss: 1.6385
Epoch 3/5
401/401 [==============================] - 1996s 5s/step - loss: 1.4619
Epoch 4/5
401/401 [==============================] - 1945s 5s/step - loss: 1.3824
Epoch 5/5
401/401 [==============================] - 1953s 5s/step - loss: 1.3309


In [91]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_5'

In [92]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [93]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            33536     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 131)            134275    
Total params: 4,106,115
Trainable params: 4,106,115
Non-trainable params: 0
_________________________________________________________________


In [94]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [97]:
print(generate_text(model, start_string=u"this movie"))

this movie in my belt Denner appear in 1295 slasher for mount expected. It is an incorbit simplight down-first pold (and get is she is Brazil'st". No paw out of 1 would probably now it was other about jokes. The movie is so excitic. God al characters, well art drink-alpoint of film change, old fact I've seen abound single s*<br />As found in cameo and knowing.<br /><br />Please people.<br /><br />Pazer: Complain Panick with Miraglia is not pretty considering this horror and included it as in.<br /><br />Now pall him, more than how CIIANAL more, literally speaking of some paceatic aiversion production and someone. The emotion is That shocking clear and gets that this, but it is not a lot beld self more like Parpers Briots ha(an abandoned lap us. Consage of many backstell & guessing one entirely nest ere is the virus one movemant Galjext's Disa". Namely point be indimantery man in the belief fears and all starÊsisent head as it's also very reasons ; later cute on 38 minute versions. We s

In [98]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [157]:
model1 = tf.keras.Sequential()
model1.add(hub_layer)
model1.add(tf.keras.layers.Dense(16, activation='relu'))
model1.add(tf.keras.layers.Dense(1))

model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense_9 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [158]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [159]:
history = model1.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 16s 542ms/step - loss: 0.6170 - accuracy: 0.5523 - val_loss: 0.5425 - val_accuracy: 0.7092
Epoch 2/10
30/30 [==============================] - 16s 532ms/step - loss: 0.3704 - accuracy: 0.9009 - val_loss: 0.3928 - val_accuracy: 0.8404
Epoch 3/10
30/30 [==============================] - 16s 528ms/step - loss: 0.2094 - accuracy: 0.9710 - val_loss: 0.3226 - val_accuracy: 0.8594
Epoch 4/10
30/30 [==============================] - 16s 532ms/step - loss: 0.1177 - accuracy: 0.9874 - val_loss: 0.3044 - val_accuracy: 0.8716
Epoch 5/10
30/30 [==============================] - 16s 532ms/step - loss: 0.0750 - accuracy: 0.9935 - val_loss: 0.3031 - val_accuracy: 0.8702
Epoch 6/10
30/30 [==============================] - 16s 527ms/step - loss: 0.0525 - accuracy: 0.9956 - val_loss: 0.3105 - val_accuracy: 0.8712
Epoch 7/10
30/30 [==============================] - 16s 528ms/step - loss: 0.0385 - accuracy: 0.9975 - val_loss: 0.3200 - val_accuracy: 0.8710

In [160]:
results = model1.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model1.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3817 - accuracy: 0.8543
loss: 0.382
accuracy: 0.854


In [179]:
test_examples_batch, test_labels_batch = next(iter(test_data.batch(2)))
train_examples_batch1,train_labels_batch1 = next(iter(train_data.batch(10)))
for x in test_examples_batch:
  print(x)

print(test_examples_batch)
x_tensor = tf.constant([generate_text(model, start_string=u"This was ")])
print(x_tensor)


tf.Tensor(b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.", shape=(), dtype=string)
tf.Tensor(b"A blackly 

In [180]:
print(train_examples_batch1)

print(x_tensor)
model1.predict(np.array(x_tensor))

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

array([[0.2484053]], dtype=float32)